## Import modules

In [1]:
from common_dirs_fns import *
from sort_seq_functions import *
import pandas as pd

# For progress bars
from tqdm.notebook import tqdm

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Import peptide count and frequency data

In [2]:
# Import peptide count and frequency data
tln_count_df = pd.read_csv(analysis_path+'tln_count_df.csv',
                                                    index_col=[0,1],
                                                    header=[0,1],
                                                    na_filter=False)

# Import the processed count and frequency data from sort-seq notebook
tln_count_df_processed = rename_unnamed(pd.read_csv(analysis_path+'tln_count_df_processed.csv',
                                                    index_col=[0],
                                                    header=[0,1],
                                                    na_filter=False))

## Set paths for storing bootstrap data

In [3]:
bootstrap_path = analysis_path+'bootstrapping/'

In [4]:
# Overwrite existing bootstrap input and output directories
# Uncomment when conducting iterations for the first time
# or if repeating the analysis

# for subfolder in ['input','output']:
#     rewrite_directory(bootstrap_path+subfolder)

## Generate 1000 simulated datasets

In [5]:
n_iter = 1000

In [7]:
# This takes approximately 4 hours to run

# Generate simulated datasets by drawing with replacement from
# underlying count data (tln_count_df)
for i in tqdm(range(n_iter)):
    random_bins = make_random_bins(tln_count_df, mixed_stats_bins)
    random_bins.to_csv(bootstrap_path+'input/'+str(i)+'.csv')

## Use simple fitting function to estimate distribution parameters for each peptide in each simulated dataset

In [8]:
# This takes over 16 hours to run
# Different iterations will have different numbers of peptides, since not all low-count peptides
# will have 10 reads in each iteration due to random sampling
minimum_reads = 10

for i in tqdm(range(n_iter)):
    # Read in input file (simulated dataset) for each iteration
    random_df = pd.read_csv(bootstrap_path+'input/'+str(i)+'.csv',
                            index_col=[0,1], header=[0,1], na_filter=False)
    
    # Process the dataframe
    random_df_processed = process_tln_count_df(random_df)
    
    # Generate stats_table for all peptides with > minimum reads using simple fitting
    stats_table = make_stats_table(random_df_processed[random_df_processed['TotalReads']>minimum_reads],
                              bootstrap_path+'output/'+'stats_table_'+str(i)+'.csv',
                              upper, lower, 'Simple')

## Create new table that contains fits from each bootstrap iteration

In [9]:
# This takes about 15 minutes to run

for i in tqdm(range(n_iter)):
    # Read in output table
    random_stats_table = pd.read_csv(bootstrap_path+'output/'+'stats_table_'+str(i)+'.csv',
                            index_col=[0], header=[0], na_filter=False)
    
    # Transform the table's columns to a multiindex to keep track of the iteration number
    random_stats_table.columns = pd.MultiIndex.from_product([[str(i)],random_stats_table.columns])
    
    try:
        # If bootstrap_stats_table already exists, join this iteration's stats_table
        # to the previous iterations'
        bootstrap_stats_table = bootstrap_stats_table.join(random_stats_table,how='outer')
    
    except NameError:
        # If bootstrap_stats_table doesn't exist (i=0), create this variable to keep
        # track of information for all iterations
        bootstrap_stats_table = random_stats_table

In [10]:
# Save bootstrap information to csv
bootstrap_stats_table.to_csv(analysis_path+'stats_table_bootstraps.csv')

## Create a table to summarize bootstrap results

In [11]:
# Calculate median, 5th percentile, and 95th percentile information for all
# peptide distribution statistics across bootstrap iterations to get approximate
# error bounds on those variables

summary_table = pd.DataFrame(index=bootstrap_stats_table.index,
                            columns=pd.MultiIndex.from_product([
                                        bootstrap_stats_table.columns.levels[1],
                                        ['Median','Pct5','Pct95']]))

for metric in tqdm(summary_table.columns.levels[0]):
    print(metric)
    summary_table[(metric,'Median')] = bootstrap_stats_table.xs(metric, level=1, axis=1).median(axis=1)
    summary_table[(metric,'Pct5')] = bootstrap_stats_table.xs(metric, level=1, axis=1).quantile(0.05, axis=1)
    summary_table[(metric,'Pct95')] = bootstrap_stats_table.xs(metric, level=1, axis=1).quantile(0.95, axis=1)

CV
Fold Change
Mean
Mu
Sigma
StdDev
Variance



In [12]:
# Determine the number of iterations where there were sufficient simulated reads (> 10)
# to calculate distribution statistics
summary_table['N_Iter'] = bootstrap_stats_table.xs('Mean', level=1, axis=1).count(axis=1)

In [13]:
# Save summary to csv
summary_table.to_csv(analysis_path+'bootstrap_summary.csv')